## PART ONE

In [1]:
import pandas as pd
import numpy as np
import string
import re
pd.options.display.max_colwidth = 500
from zipfile import ZipFile
import os

import warnings 
warnings.filterwarnings(action="ignore")

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer

import langdetect
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer

#### 1. Read and Analyse Dataset

In [2]:
#1) Clearly write outcome of data analysis(Minimum 2 points)

In [3]:
#unzip
with ZipFile("blogs.zip") as file:
    file.extractall()

In [4]:
ls

 Volume in drive C is OSDisk
 Volume Serial Number is C0A0-881F

 Directory of C:\Users\nirajesh\Desktop\GL\NLP\Project

08/06/2022  09:23 PM    <DIR>          .
08/06/2022  09:23 PM    <DIR>          ..
07/21/2022  08:36 AM    <DIR>          .ipynb_checkpoints
08/06/2022  08:33 AM                96 2
08/06/2022  08:20 AM       303,646,744 blogs.zip
08/06/2022  09:31 PM       800,419,647 blogtext.csv
08/06/2022  08:18 AM             6,520 GL Bot (1).json
08/06/2022  09:23 PM            77,857 NLP Project -1.ipynb
07/21/2022  08:23 AM           387,583 NLP Project 1.pdf
08/06/2022  08:18 AM           296,090 NLP-1_Problem Statement.pdf
07/21/2022  08:23 AM         5,647,880 Text chat bot testing video.mov
               8 File(s)  1,110,482,417 bytes
               3 Dir(s)  340,518,187,008 bytes free


In [5]:
#read the data into df
bloggers_df =pd.read_csv("blogtext.csv")

In [6]:
# shape of the dataset
bloggers_df.shape

(681284, 7)

In [7]:
#sample data
bloggers_df.head(10)

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages, and 4.5 MB of .pdf files) Now i have to wait untill our team leader has processed it and learns html."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewes van der Laag urlLink mail Ruiyu Xie urlLink mail Bryan Aaldering (me) urlLink mail
2,2059027,male,15,Student,Leo,"12,May,2004","In het kader van kernfusie op aarde: MAAK JE EIGEN WATERSTOFBOM How to build an H-Bomb From: ascott@tartarus.uwa.edu.au (Andrew Scott) Newsgroups: rec.humor Subject: How To Build An H-Bomb (humorous!) Date: 7 Feb 1994 07:41:14 GMT Organization: The University of Western Australia Original file dated 12th November 1990. Seemed to be a transcript of a 'Seven Days' article. Poorly formatted and corrupted. I have added the text between 'examine under a microscope' and 'malleable, ..."
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004","Thanks to Yahoo!'s Toolbar I can now 'capture' the URLs of popups...which means now I can show you some cool links to Korean Pop (K-Pop) audio and video without the need to relate instructions like: 'go to the site, click on the pop-audio button then choose...'. So, without further ado here is the link to 24-hour K-Pop urlLink audio and the urlLink video streaming. Enjoy."
5,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","I had an interesting conversation with my Dad this morning. We were talking about where Koreans put their money. Invariably, they have a lot of real estate and cash. (Cash would include short term investments under one year as well as savings accounts.) The reason? Real estate makes money here. A lot of money. I've seen surveys of Seoul real estate rising about 10-15% PER YEAR for long stretches, even after taking into account the 1997 Crisis (referred to as the IMF crisi..."
6,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","Somehow Coca-Cola has a way of summing up things so well. In the early 1970s they had as their flagship jingle 'I'd Like to Buy the World a Coke' (to the tune of 'I'd Like to Teach the World to Sing') that pretty much summed up the post-Woodstock era so well. It didn't add much to sales, but it was a catchy tune. In Korea Coke's theme is urlLink Stop Thinking. Feel it. which pretty much sums up a lot about Korea and Koreans. (Look at how relaxed that couple is, now that th..."
7,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","If anything, Korea is a country of extremes. Everything here seems fad-based. I think it may come from Korea's history. It has been invaded a reported 700 times over the years, and each time they got independence I imagine they had to move quickly to get to the next level before the next war or occupation. Lately (well, not really lately...in 1945) the Japanese Occupation ended. Then the Korean War occurred from 1950-3. After that there was turmoil, but in 1961 Park Chung ..."
8,3581210,male,33,InvestmentBanking,Aquarius,"10,June,2004","Take a read of this news article from urlLink JoongAng Ilbo about North Korea's opinion on the scaling down of troops in Seoul and Korea (specifically, 3,600 troops are heading to Iraq and by the end of 2005 the current total of 37,000 should drop to about 24,500). North blasts U.S. troop cuts North Korea again blasted the United States yesterday for what it calls 'an increase of U.S. defense capabilities' after the planned relocation of 3,600 U.S. troops in South Korea to I..."
9,3581210,male,33,InvestmentBanking,Aquarius,"09,June,2004","I surf the English news sites a lot looking for tidbits on Korea and how foreigners (like me) view the 'Hermit Kingdom' but also as a way to keep up with this fast-moving place. Sometimes, though, one needs to check the veracity of the figures put in the papers...especially the local ones. Here are two examples of how the English version of the Kor

In [8]:
bloggers_df.id.nunique()

19320

In [9]:
#datatypes
bloggers_df.dtypes

id         int64
gender    object
age        int64
topic     object
sign      object
date      object
text      object
dtype: object

In [10]:
object_col=[col for col in bloggers_df.columns if bloggers_df[col].dtype == 'O' ]
bloggers_df[object_col].describe()

,gender,topic,sign,date,text
count,681284,681284,681284,681284,681284
unique,2,40,12,2616,611652
top,male,indUnk,Cancer,"02,August,2004",urlLink
freq,345193,251015,65048,16544,445


- Most of the Posts from bloggers related indUnk topic
- There are total 40 topics in the dataset

In [11]:
bloggers_df[bloggers_df['topic'] == 'indUnk']['text'].head(1)

74                      O= optimist P= pessimist  My argument with myself:  P: Nooooo! Stop thinking about him! O: Why? He's my boyfriend. I'm allowed to. P: You're obsessing! O: What?! No I'm not. I just like him a lot. P: You're crazy if you think it's going to work! O: Ok... Then I'm crazy! P: So you think it will work out between you two? Do you think he's 'the one'? O: Yes and possibly. P: Insane! I can't believe you! You know what is going to happen! The same thing that always happens. You w...
Name: text, dtype: object

In [12]:
bloggers_df['gender'].value_counts()

male      345193
female    336091
Name: gender, dtype: int64

- Almost Equal number of male and female bloggers

In [13]:
numerical_col=[col for col in bloggers_df.columns if bloggers_df[col].dtype != 'O' ]
bloggers_df[numerical_col].describe().apply(lambda s: s.apply('{0:.2f}'.format))

,id,age
count,681284.00,681284.00
mean,2397802.40,23.93
std,1247722.70,7.79
min,5114.00,13.00
25%,1239610.00,17.00
50%,2607577.00,24.00
75%,3525660.00,26.00
max,4337650.00,48.00


Most of the bloggers with age between 24 to 26

In [14]:
bloggers_df['topic'].value_counts()

indUnk                     251015
Student                    153903
Technology                  42055
Arts                        32449
Education                   29633
Communications-Media        20140
Internet                    16006
Non-Profit                  14700
Engineering                 11653
Law                          9040
Publishing                   7753
Science                      7269
Government                   6907
Consulting                   5862
Religion                     5235
Fashion                      4851
Marketing                    4769
Advertising                  4676
BusinessServices             4500
Banking                      4049
Chemicals                    3928
Telecommunications           3891
Accounting                   3832
Military                     3128
Museums-Libraries            3096
Sports-Recreation            3038
HumanResources               3010
RealEstate                   2870
Transportation               2326
Manufacturing 

In [15]:
bloggers_df['sign'].value_counts()

Cancer         65048
Aries          64979
Taurus         62561
Libra          62363
Virgo          60399
Scorpio        57161
Pisces         54053
Leo            53811
Gemini         51985
Sagittarius    50036
Aquarius       49687
Capricorn      49201
Name: sign, dtype: int64

In [16]:
#B) Clean the Structured Data

In [17]:
#i) Missing value analysis and imputation.

In [18]:
bloggers_df.isna().sum()

id        0
gender    0
age       0
topic     0
sign      0
date      0
text      0
dtype: int64

 No nulls present in the dataset

In [19]:
#ii.)  Eliminate Non-English textual data

In [20]:
bloggers_df=bloggers_df.sample(5000)

In [21]:
from langdetect import detect

def detect_english(text):
    try:
        return detect(text) == 'en'
    except:
        return False

In [22]:
bloggers_df=bloggers_df[bloggers_df['text'].apply(detect_english)]

In [23]:
#2)Preprocess unstructured data to make it consumable for model training.

In [24]:
#A) Eliminate All special Characters and Numbers

In [25]:
bloggers_df['clean_text'] = bloggers_df['text'].apply(lambda x:re.sub('[^a-zA-Z]', ' ', x))

In [26]:
bloggers_df.head()

,id,gender,age,topic,sign,date,text,clean_text
639337,3578628,male,16,Education,Taurus,"16,June,2004","I would like to take this opportunity to thank Spencer for staying up to 12:30 tonight to explain sports to me. I would also like to take this time to thank everyone who has taken the time to explain sports to me, like Josh and David. Working with these guys has been really nice and a true learning experience in all fields. I know...you're all like 'It's no big deal. We're cool with it,' but I am very sentimental and appreciate everything, so I like to thank people. I know ...",I would like to take this opportunity to thank Spencer for staying up to tonight to explain sports to me I would also like to take this time to thank everyone who has taken the time to explain sports to me like Josh and David Working with these guys has been really nice and a true learning experience in all fields I know you re all like It s no big deal We re cool with it but I am very sentimental and appreciate everything so I like to thank people I know ...
191552,3872536,male,17,Student,Gemini,"01,June,2004","Another tired but meaningful day. Today went school as usual to do the door gifts for the Leo camp. Do from 11 to about 4+ ba. We actually managed to finish all the doorgift by today lor, all the painting n writing of words. So satisfied with our product from far. Coz the doorgifts look so colourful n nice from far, but when one actually moves closer, it's like you'll see many cracks on it lor. So I was suggesting that we'll show the guests how nice our gifts are but when distributin...",Another tired but meaningful day Today went school as usual to do the door gifts for the Leo camp Do from to about ba We actually managed to finish all the doorgift by today lor all the painting n writing of words So satisfied with our product from far Coz the doorgifts look so colourful n nice from far but when one actually moves closer it s like you ll see many cracks on it lor So I was suggesting that we ll show the guests how nice our gifts are but when distributin...
323628,3761156,female,15,indUnk,Pisces,"01,July,2004","First of all . . . Lauren, tomorrow you WILL! I really wanna go to HOM! So . . . I got a letter from Mr. Suppon. He is one of my favorite teachers, if not my favorite teacher. I love photo way much, but I am pretty darn good at screwing things up in that department. Cheating is bad, especially if you are a really bad cheater and you end up making it look like you were cheating even more than you were before. Yeah, so, that made no sense. However, I did it. I muddled up coppying t...",First of all Lauren tomorrow you WILL I really wanna go to HOM So I got a letter from Mr Suppon He is one of my favorite teachers if not my favorite teacher I love photo way much but I am pretty darn good at screwing things up in that department Cheating is bad especially if you are a really bad cheater and you end up making it look like you were cheating even more than you were before Yeah so that made no sense However I did it I muddled up coppying t...
495536,3451935,male,17,Banking,Aries,"25,May,2004",urlLink duncy-poos and ali-poos. duncy-poos lookin a bit concerned about something&nbsp; urlLink,urlLink duncy poos and ali poos duncy poos lookin a bit concerned about something nbsp urlLink
524919,1000866,female,17,Student,Libra,"04,January,2004",Note to Catie: You're a moron. Listen to yourself when you say go to bed at a reasonable hour. Another note to Catie: Hit yourself over the head tomorrow morning. Note to Catie's head: Watch out. Catie's going to hit you tomorrow morning. Final note to self: Stop writing notes to yourself you ignoramus. You're going to scare people away.,Note to Catie You re a moron Listen to yourself when you say go to bed at a reasonable hour Another note to Catie Hit yourself over the head tomorrow morning Note to Catie s head Watch out Catie s going to hit you tomorrow morning Final note to self Stop writing notes to yourself y

In [27]:
#B) Lowercase all textual data 

In [28]:
bloggers_df['clean_text']=bloggers_df['clean_text'].apply(lambda x:x.lower())

In [29]:
bloggers_df.head()

,id,gender,age,topic,sign,date,text,clean_text
639337,3578628,male,16,Education,Taurus,"16,June,2004","I would like to take this opportunity to thank Spencer for staying up to 12:30 tonight to explain sports to me. I would also like to take this time to thank everyone who has taken the time to explain sports to me, like Josh and David. Working with these guys has been really nice and a true learning experience in all fields. I know...you're all like 'It's no big deal. We're cool with it,' but I am very sentimental and appreciate everything, so I like to thank people. I know ...",i would like to take this opportunity to thank spencer for staying up to tonight to explain sports to me i would also like to take this time to thank everyone who has taken the time to explain sports to me like josh and david working with these guys has been really nice and a true learning experience in all fields i know you re all like it s no big deal we re cool with it but i am very sentimental and appreciate everything so i like to thank people i know ...
191552,3872536,male,17,Student,Gemini,"01,June,2004","Another tired but meaningful day. Today went school as usual to do the door gifts for the Leo camp. Do from 11 to about 4+ ba. We actually managed to finish all the doorgift by today lor, all the painting n writing of words. So satisfied with our product from far. Coz the doorgifts look so colourful n nice from far, but when one actually moves closer, it's like you'll see many cracks on it lor. So I was suggesting that we'll show the guests how nice our gifts are but when distributin...",another tired but meaningful day today went school as usual to do the door gifts for the leo camp do from to about ba we actually managed to finish all the doorgift by today lor all the painting n writing of words so satisfied with our product from far coz the doorgifts look so colourful n nice from far but when one actually moves closer it s like you ll see many cracks on it lor so i was suggesting that we ll show the guests how nice our gifts are but when distributin...
323628,3761156,female,15,indUnk,Pisces,"01,July,2004","First of all . . . Lauren, tomorrow you WILL! I really wanna go to HOM! So . . . I got a letter from Mr. Suppon. He is one of my favorite teachers, if not my favorite teacher. I love photo way much, but I am pretty darn good at screwing things up in that department. Cheating is bad, especially if you are a really bad cheater and you end up making it look like you were cheating even more than you were before. Yeah, so, that made no sense. However, I did it. I muddled up coppying t...",first of all lauren tomorrow you will i really wanna go to hom so i got a letter from mr suppon he is one of my favorite teachers if not my favorite teacher i love photo way much but i am pretty darn good at screwing things up in that department cheating is bad especially if you are a really bad cheater and you end up making it look like you were cheating even more than you were before yeah so that made no sense however i did it i muddled up coppying t...
495536,3451935,male,17,Banking,Aries,"25,May,2004",urlLink duncy-poos and ali-poos. duncy-poos lookin a bit concerned about something&nbsp; urlLink,urllink duncy poos and ali poos duncy poos lookin a bit concerned about something nbsp urllink
524919,1000866,female,17,Student,Libra,"04,January,2004",Note to Catie: You're a moron. Listen to yourself when you say go to bed at a reasonable hour. Another note to Catie: Hit yourself over the head tomorrow morning. Note to Catie's head: Watch out. Catie's going to hit you tomorrow morning. Final note to self: Stop writing notes to yourself you ignoramus. You're going to scare people away.,note to catie you re a moron listen to yourself when you say go to bed at a reasonable hour another note to catie hit yourself over the head tomorrow morning note to catie s head watch out catie s going to hit you tomorrow morning final note to self stop writing notes to yourself y

In [30]:
#C)Remove all Stopwords

In [31]:
stop_words = stopwords.words('english')

def remove_stopwords(text):
    return " ".join([word for word in str(text).split() if word not in stop_words])

bloggers_df['clean_text']=bloggers_df['clean_text'].apply(lambda x:remove_stopwords(x))

In [32]:
#D) Remove all extra white spaces

In [33]:
bloggers_df['clean_text']=bloggers_df['clean_text'].apply(lambda x:re.sub(r' +',' ',x))

In [34]:
bloggers_df['clean_text']=bloggers_df['clean_text'].apply(lambda x:x.strip())

In [35]:
# considering few samples for the model development

bloggers_df.head()

,id,gender,age,topic,sign,date,text,clean_text
639337,3578628,male,16,Education,Taurus,"16,June,2004","I would like to take this opportunity to thank Spencer for staying up to 12:30 tonight to explain sports to me. I would also like to take this time to thank everyone who has taken the time to explain sports to me, like Josh and David. Working with these guys has been really nice and a true learning experience in all fields. I know...you're all like 'It's no big deal. We're cool with it,' but I am very sentimental and appreciate everything, so I like to thank people. I know ...",would like take opportunity thank spencer staying tonight explain sports would also like take time thank everyone taken time explain sports like josh david working guys really nice true learning experience fields know like big deal cool sentimental appreciate everything like thank people know half guy serious guys funny note prove wrong lol keep pushing learn new things like said wanting thank everyone get strange feeling done sorry get bitchy slap tell move get picture
191552,3872536,male,17,Student,Gemini,"01,June,2004","Another tired but meaningful day. Today went school as usual to do the door gifts for the Leo camp. Do from 11 to about 4+ ba. We actually managed to finish all the doorgift by today lor, all the painting n writing of words. So satisfied with our product from far. Coz the doorgifts look so colourful n nice from far, but when one actually moves closer, it's like you'll see many cracks on it lor. So I was suggesting that we'll show the guests how nice our gifts are but when distributin...",another tired meaningful day today went school usual door gifts leo camp ba actually managed finish doorgift today lor painting n writing words satisfied product far coz doorgifts look colourful n nice far one actually moves closer like see many cracks lor suggesting show guests nice gifts distributing stuff lights actually see home like already stepped busi lor juz lame lah enjoy much gifts lol eat lotsa stuff lunch coz damn hungry rojak n mince meat noodle lor tat go tm shop xxx birthday p...
323628,3761156,female,15,indUnk,Pisces,"01,July,2004","First of all . . . Lauren, tomorrow you WILL! I really wanna go to HOM! So . . . I got a letter from Mr. Suppon. He is one of my favorite teachers, if not my favorite teacher. I love photo way much, but I am pretty darn good at screwing things up in that department. Cheating is bad, especially if you are a really bad cheater and you end up making it look like you were cheating even more than you were before. Yeah, so, that made no sense. However, I did it. I muddled up coppying t...",first lauren tomorrow really wanna go hom got letter mr suppon one favorite teachers favorite teacher love photo way much pretty darn good screwing things department cheating bad especially really bad cheater end making look like cheating even yeah made sense however muddled coppying end simple paper turning wrong paper looking like total lying idiot suppose deserve though lame end year incomplete favorite class extraordinarily lame stupid paper fourth possibly fifth time compensate getting ...
495536,3451935,male,17,Banking,Aries,"25,May,2004",urlLink duncy-poos and ali-poos. duncy-poos lookin a bit concerned about something&nbsp; urlLink,urllink duncy poos ali poos duncy poos lookin bit concerned something nbsp urllink
524919,1000866,female,17,Student,Libra,"04,January,2004",Note to Catie: You're a moron. Listen to yourself when you say go to bed at a reasonable hour. Another note to Catie: Hit yourself over the head tomorrow morning. Note to Catie's head: Watch out. Catie's going to hit you tomorrow morning. Final note to self: Stop writing notes to yourself you ignoramus. You're going to scare people away.,note catie moron listen say go bed reasonable hour another note catie hit head tomorrow morning note catie head watch catie going hit tomorrow morning final note self stop writing notes ignoramus going scare people away


In [36]:
#3) Build a base Classification model

In [37]:
#split into the X and y labels
X = bloggers_df['clean_text']
y = bloggers_df['topic']

In [38]:
# Train and test split

In [39]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.20,random_state=43)

In [40]:
print('Training samples count:',X_train.shape[0])
print('Testing samples count:',X_test.shape[0])

Training samples count: 3818
Testing samples count: 955


In [41]:
# Vectorizer
vectorizer = CountVectorizer(ngram_range=(1,2))

vectorizer.fit(X_train)

X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [42]:
#3) Design, train, tune and test the best text classifier.

In [44]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()


lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
#y_proba_ovr = ovr.predict_proba(X_test_dtm)


In [ ]:
from sklearn.metrics import accuracy_score,f1_score,average_precision_score,recall_score

print('Accuracy score: ', accuracy_score(y_test, y_pred))
print('F1 score: Micro', f1_score(y_test, y_pred, average='micro'))
print('Average precision score: Micro', average_precision_score(y_test, y_pred, average='micro'))
print('Average recall score: Micro', recall_score(y_test, y_pred, average='micro'))

In [ ]:
for i in range(5):
    print('Title:\t{}\nTrue labels:\t{}\nPredicted labels:\t{}\n\n'.format(
        X_test[i],
        ','.join(y_test_inversed[i]),
        ','.join(pred_inversed[i])
    ))